##Cleaning Tweets

In [ ]:
# Install package 
%pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.5 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Imports 

import pandas as pd
import numpy as np
from collections import Counter
import nltk
import re
import string

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from prettytable import PrettyTable

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

import chardet
import contractions

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import bigrams
from nltk import ngrams

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def Clean(tweets):

    tweetsclean = tweets.str.lower() 
    
    def RemoveURLs(text):
        return " ".join(re.sub(r'(http|www)\S+', "", text).split()) 
    
    tweetsclean = tweetsclean.apply(RemoveURLs)
    
    def RemoveMentions(text):
        return " ".join(re.sub(r'@\S+', "", text).split())

    tweetsclean = tweetsclean.apply(RemoveMentions)
    
    def RemoveNonAscii(text):
        return "".join(w for w in text if ord(w) < 128) 
    tweetsclean = tweetsclean.apply(RemoveNonAscii)
    
    # Expand contractions
    def ExpandContractions(text):
        Expanded = contractions.fix(text)
        return Expanded

    tweetsclean = tweetsclean.apply(ExpandContractions)


    # Tokenize
    def Tokenizer(text):
        tokens = nltk.word_tokenize(text)
        tokenized = ( " ".join([w for w in tokens])) 
        return tokenized

    tweetsclean = tweetsclean.apply(Tokenizer)
    
    puncts = string.punctuation
    nltkstp = set(stopwords.words('english'))
    Stopwords = nltkstp.union(STOPWORDS)
    
    def PunctsAndStopRemover(text):
        return " ".join([w for w in text.split() if (w not in Stopwords and w not in puncts)])

    tweetsclean = tweetsclean.apply(PunctsAndStopRemover)

    # lemmatize with POS
    def POS(text):
        tagged = nltk.pos_tag(text.split())
        return tagged

    tags = tweetsclean.apply(POS)

    def WordNet(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None

    def GetWT(text):  
        WordNets = []   
        for t in text:
            WordPos = (t[0], WordNet(t[1]))
            WordNets.append(WordPos)
        return WordNets

    tags = tags.apply(GetWT)

    lemmatizer = WordNetLemmatizer()

    def POSLEM(text):
        sent = ""
        for w, t in text:
            if t is None:
                word = "".join(lemmatizer.lemmatize(w))
            else:       
                word = "".join(lemmatizer.lemmatize(w,t))
            sent = sent + " " + word
        return sent

    tweetsclean = tags.apply(POSLEM)

    return tweetsclean

In [ ]:
UsersTweets = pd.read_csv('Activist1.csv')
UsersTweets.head()

,Unnamed: 0,Timestamp,Tweet,User ID,Retweets,Replies,Likes,Quotes
0,0,2023-04-09 01:42:51+00:00,RT @DaynaReggero: W @Mzozane @zhaabowekwe @Ind...,1.644880e+18,23,0,0,0
1,1,2023-04-08 03:55:02+00:00,RT @Mzozane: @RBC SECURITY called the cops on ...,1.644550e+18,9,0,0,0
2,2,2023-04-07 22:53:22+00:00,Inclusive = racially segregated through a colo...,1.644470e+18,9,1,16,0
3,3,2023-04-07 22:48:15+00:00,Shareholders heard us loud and clear and nearl...,1.644470e+18,3,0,11,0
4,4,2023-04-07 22:48:15+00:00,"We will not be silenced. Despite insult, racis...",1.644470e+18,3,1,11,0


In [ ]:
UsersTweets['CleanTweet'] = Clean(UsersTweets['Tweet'])

UsersTweets[['Tweet', 'CleanTweet']].head(10)

,Tweet,CleanTweet
0,RT @DaynaReggero: W @Mzozane @zhaabowekwe @Ind...,rt w saskatoon shareholder meeting stoplng in...
1,RT @Mzozane: @RBC SECURITY called the cops on ...,rt security call cop try arrested basis misch...
2,Inclusive = racially segregated through a colo...,inclusive racially segregate colour pas syste...
3,Shareholders heard us loud and clear and nearl...,shareholder hear u loud clear nearly third vo...
4,"We will not be silenced. Despite insult, racis...",silence despite insult racism segregation ind...
5,"Beyond the insult of segregation, laying hands...",beyond insult segregation lay hand chiefs reg...
6,This was a calculated move. Our Chiefs were me...,calculate move chief meet security police lay...
7,RBC forced Indigenous and Black voices to be s...,rbc force indigenous black voice segregate se...
8,"Outside, we gathered to rally in support of th...",outside gather rally support inside sing chan...
9,"Their names: Eve Saint, Vanessa Gray, Auntie J...",name eve saint vanessa gray auntie janet joce...


In [ ]:
# Drop rows with missing values 
print(UsersTweets.shape)
UsersTweets.dropna(subset = ['CleanTweet'], inplace=True) 
print(UsersTweets.shape)

(2168, 9)
(2168, 9)


In [ ]:
# To drop rows with missing "string" values, you need to first convert them to numpy nan objects
UsersTweets['CleanTweet'].replace('', np.nan, inplace=True)
UsersTweets.dropna(subset = ['CleanTweet'], inplace=True)
print(UsersTweets.shape)

(2127, 9)


In [ ]:
# Drop original tweet column to obtain final dataset 
UsersTweets.drop('Tweet', axis = 1, inplace=True)
UsersTweets.head(10)

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet
0,0,2023-04-09 01:42:51+00:00,1.644880e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...
5,5,2023-04-07 22:48:14+00:00,1.644470e+18,4,1,11,0,beyond insult segregation lay hand chiefs reg...
6,6,2023-04-07 22:48:14+00:00,1.644470e+18,4,1,10,0,calculate move chief meet security police lay...
7,7,2023-04-07 22:48:13+00:00,1.644470e+18,3,1,10,0,rbc force indigenous black voice segregate se...
8,8,2023-04-07 22:48:11+00:00,1.644470e+18,2,1,11,0,outside gather rally support inside sing chan...
9,9,2023-04-07 22:48:11+00:00,1.644470e+18,4,1,11,0,name eve saint vanessa gray auntie janet joce...


In [ ]:
UsersTweets.to_csv('CleanTweets1.csv', index = False)

### Sentiment Analysis using Happy Transformer 

In [ ]:
%pip install happytransformer

In [ ]:
from happytransformer import HappyTextClassification

In [ ]:
# Initialize 
# The model choosen is cardiffnlp/twitter-roberta-base-sentiment-latest
# labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive
# Trained on tweet eval dataset 

happy_tc1 = HappyTextClassification(model_type="ROBERTA", model_name="cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load clean tweets 
ct1 = pd.read_csv('/content/CleanTweets1.csv') 
ct1.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet
0,0,2023-04-09 01:42:51+00:00,1.644880e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...


In [ ]:
sentiment =[AssertionError]
for i in ct1['CleanTweet']:
  label = happy_tc1.classify_text(i).label
  sentiment.append(label)


In [ ]:
l = pd.DataFrame(sentiment, columns = ['label'])
l.value_counts()

label                   
neutral                     1430
negative                     498
positive                     199
<class 'AssertionError'>       1
dtype: int64

In [ ]:
# Add labels to the orginal dataframe 
ct1['Action'] = sentiment[1:]

In [ ]:
ct1.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action
0,0,2023-04-09 01:42:51+00:00,1.644880e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...,neutral
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...,neutral
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...,neutral
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...,neutral


In [ ]:
ct1.to_csv('CleanLabeledTweets1.csv', index = False)

In [ ]:
# Load dataset of clean labelled tweets 

data = pd.read_csv('/content/CleanLabeledTweets1.csv')
data.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action
0,0,2023-04-09 01:42:51+00:00,1.644880e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...,neutral
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...,neutral
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...,neutral
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...,neutral


Mapping Retweets to states 

1.   High - 4
2.   Medium_high - 3
3.   Medium_low - 2
3.   Low - 1 




In [ ]:
def RT_to_state(x):
    if x >= 100:
        # High 
        return 4
    if x < 100 and x > 10 :
        # Medium_high
        return 3
    if x <= 10 and x > 1:
        # Medium_low
        return 2
    if x <= 1 and x >= 0:
        # Low
        return 1

In [ ]:
# Assign States
data['State'] = data['Retweets'].apply(RT_to_state).shift(-1, fill_value=1) 
data.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State
0,0,2023-04-09 01:42:51+00:00,1.644880e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral,2
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...,neutral,2
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...,neutral,2
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...,neutral,2
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...,neutral,2


In [ ]:
# Check distribution 
data['State'].value_counts()

2    839
3    726
1    354
4    208
Name: State, dtype: int64

In [ ]:
# Function for next state 
# Use the shift function 


data['Next_State'] = data['State'].shift(1) # Shifts the states columns but as floats 

data.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State
0,0,2023-04-09 01:42:51+00:00,1.644880e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral,2,NaN
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...,neutral,2,2.0
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...,neutral,2,2.0
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...,neutral,2,2.0
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...,neutral,2,2.0


In [ ]:
data = data.drop(0)
data.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...,neutral,2,2.0
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...,neutral,2,2.0
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...,neutral,2,2.0
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...,neutral,2,2.0
5,5,2023-04-07 22:48:14+00:00,1.644470e+18,4,1,11,0,beyond insult segregation lay hand chiefs reg...,negative,2,2.0


In [ ]:
def NextState_to_Reward(x):
    if x == 4:
        # High 
        return 100
    if x == 3  :
        # Medium_high
        return 25
    if x == 2:
        # Medium_low
        return 10
    if x == 1:
        # Low
        return -1

In [ ]:
data['Reward'] = data['Next_State'].apply(NextState_to_Reward)
data.head(10)

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State,Reward
1,1,2023-04-08 03:55:02+00:00,1.644550e+18,9,0,0,0,rt security call cop try arrested basis misch...,neutral,2,2.0,10
2,2,2023-04-07 22:53:22+00:00,1.644470e+18,9,1,16,0,inclusive racially segregate colour pas syste...,neutral,2,2.0,10
3,3,2023-04-07 22:48:15+00:00,1.644470e+18,3,0,11,0,shareholder hear u loud clear nearly third vo...,neutral,2,2.0,10
4,4,2023-04-07 22:48:15+00:00,1.644470e+18,3,1,11,0,silence despite insult racism segregation ind...,neutral,2,2.0,10
5,5,2023-04-07 22:48:14+00:00,1.644470e+18,4,1,11,0,beyond insult segregation lay hand chiefs reg...,negative,2,2.0,10
6,6,2023-04-07 22:48:14+00:00,1.644470e+18,4,1,10,0,calculate move chief meet security police lay...,neutral,2,2.0,10
7,7,2023-04-07 22:48:13+00:00,1.644470e+18,3,1,10,0,rbc force indigenous black voice segregate se...,negative,2,2.0,10
8,8,2023-04-07 22:48:11+00:00,1.644470e+18,2,1,11,0,outside gather rally support inside sing chan...,positive,2,2.0,10
9,9,2023-04-07 22:48:11+00:00,1.644470e+18,4,1,11,0,name eve saint vanessa gray auntie janet joce...,neutral,2,2.0,10
10,10,2023-04-07 22:48:10+00:00,1.644470e+18,5,1,11,0,wetsuwetenstrong hereditary chief matriarchs ...,negative,3,2.0,10


In [ ]:
data_reverse = data[::-1].reset_index(drop=True)
data_reverse.head()


,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State,Reward
0,2167,2021-04-09 05:07:14+00:00,1.380000e+18,1,0,0,0,rt sign pledge learn bank alternative link bi...,neutral,1,1.0,-1
1,2166,2021-04-09 05:07:30+00:00,1.380000e+18,2,0,0,0,rt smart money fossil fuel hey getsmart fossi...,neutral,1,2.0,10
2,2165,2021-04-09 05:07:49+00:00,1.380000e+18,3,0,0,0,rt `` ... climate policy leave corporate powe...,neutral,2,2.0,10
3,2164,2021-04-09 05:08:13+00:00,1.380000e+18,1,0,0,0,rt `` carbon tax constrain collective imagina...,neutral,2,1.0,-1
4,2163,2021-04-09 05:10:22+00:00,1.380000e+18,2,0,0,0,rt big news,neutral,1,2.0,10


In [ ]:
# Generate Final dataset 

dataset = data_reverse.drop(columns=['Timestamp',	'Retweets',	'Replies',	'Likes',	'Quotes',	'CleanTweet'])
dataset.head()

,Unnamed: 0,User ID,Action,State,Next_State,Reward
0,2167,1.380000e+18,neutral,1,1.0,-1
1,2166,1.380000e+18,neutral,1,2.0,10
2,2165,1.380000e+18,neutral,2,2.0,10
3,2164,1.380000e+18,neutral,2,1.0,-1
4,2163,1.380000e+18,neutral,1,2.0,10


In [ ]:
# Reorder columns 

Dataset_final = pd.DataFrame({'State' : dataset['State'], 'Action' : dataset['Action'], 'Reward' : dataset['Reward'], 'Next_State': dataset['Next_State']})
Dataset_final.head()

,State,Action,Reward,Next_State
0,1,neutral,-1,1.0
1,1,neutral,10,2.0
2,2,neutral,10,2.0
3,2,neutral,-1,1.0
4,1,neutral,10,2.0


In [ ]:
# Save Dataset 
Dataset_final.to_csv('Final_Datasetnew.csv', index = False)

## Second Activist Tweets

In [ ]:
UsersTweets_2 = pd.read_csv('Activist2.csv')
UsersTweets_2.head()

,Unnamed: 0,Timestamp,Tweet,User ID,Retweets,Replies,Likes,Quotes
0,0,2023-04-09 23:01:03+00:00,RT @DaynaReggero: W @Mzozane @zhaabowekwe @Ind...,1.645200e+18,23,0,0,0
1,1,2023-04-09 21:26:41+00:00,🥳💚🥳a NEW group of reasonable students &amp; yo...,1.645180e+18,12,0,17,0
2,2,2023-04-09 21:16:45+00:00,RT @ClimateDefiance: We just had our official ...,1.645170e+18,18,0,0,0
3,3,2023-04-09 21:16:12+00:00,RT @ClimateDefiance: YOUNG PEOPLE SHOULD NOT H...,1.645170e+18,108,0,0,0
4,4,2023-04-09 21:16:04+00:00,RT @XRebellionLA: This is gonna be big! But we...,1.645170e+18,5,0,0,0


In [ ]:
UsersTweets_2['CleanTweet'] = Clean(UsersTweets_2['Tweet'])

UsersTweets_2[['Tweet', 'CleanTweet']].head(10)

,Tweet,CleanTweet
0,RT @DaynaReggero: W @Mzozane @zhaabowekwe @Ind...,rt w saskatoon shareholder meeting stoplng in...
1,🥳💚🥳a NEW group of reasonable students &amp; yo...,new group reasonable student amp youth rise c...
2,RT @ClimateDefiance: We just had our official ...,rt official launch join 100 community member
3,RT @ClimateDefiance: YOUNG PEOPLE SHOULD NOT H...,rt young people risk arrest politician someth...
4,RT @XRebellionLA: This is gonna be big! But we...,rt go big need make even big eye event let u ...
5,RT @ClimateDefiance: For too long we have been...,rt long treat disposable asked wait ask reaso...
6,"RT @ClimateDefiance: We are a brand-new, youth...",rt brand-new youth-led collective 20 day bloc...
7,RT @skyerich_luna: ⁦@bctf⁩ ⁦@FPSE⁩ ⁦@ZeroOilCa...,rt
8,RT @Docs4Divestment: @cjuoft @QuitRbc @VicColl...,rt applaud work towards fossilfueldivestment ab
9,RT @Chantalartich: RBC est la « Pétro-banque d...,rt rbc est la ptro-banque du canada elle prte...


In [ ]:
# Drop rows with missing values 
print(UsersTweets_2.shape)
UsersTweets_2.dropna(subset = ['CleanTweet'], inplace=True) 
print(UsersTweets_2.shape)

(4449, 9)
(4449, 9)


In [ ]:
# To drop rows with missing "string" values, you need to first convert them to numpy nan objects
UsersTweets_2['CleanTweet'].replace('', np.nan, inplace=True)
UsersTweets_2.dropna(subset = ['CleanTweet'], inplace=True)
print(UsersTweets_2.shape)

(4433, 9)


In [ ]:
# Drop original tweet column to obtain final dataset 
UsersTweets_2.drop('Tweet', axis = 1, inplace=True)
UsersTweets_2.head(10)

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet
0,0,2023-04-09 23:01:03+00:00,1.645200e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...
5,5,2023-04-09 21:15:26+00:00,1.645170e+18,16,0,0,0,rt long treat disposable asked wait ask reaso...
6,6,2023-04-09 20:07:50+00:00,1.645160e+18,359,0,0,0,rt brand-new youth-led collective 20 day bloc...
7,7,2023-04-09 14:56:56+00:00,1.645080e+18,2,0,0,0,rt
8,8,2023-04-09 14:56:19+00:00,1.645080e+18,1,0,0,0,rt applaud work towards fossilfueldivestment ab
9,9,2023-04-09 00:09:24+00:00,1.644850e+18,2,0,0,0,rt rbc est la ptro-banque du canada elle prte...


In [ ]:
UsersTweets_2.to_csv('CleanTweets2.csv', index = False)

In [ ]:
# Load clean tweets 
ct2 = pd.read_csv('/content/CleanTweets2.csv') 
ct2.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet
0,0,2023-04-09 23:01:03+00:00,1.645200e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...


In [ ]:
sentiment =[AssertionError]
for i in ct2['CleanTweet']:
  label = happy_tc1.classify_text(i).label
  sentiment.append(label)


In [ ]:
l2 = pd.DataFrame(sentiment, columns = ['label'])
l2.value_counts()

label                   
neutral                     2972
negative                    1189
positive                     272
<class 'AssertionError'>       1
dtype: int64

In [ ]:
# Add labels to the orginal dataframe 
ct2['Action'] = sentiment[1:]

In [ ]:
ct2.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action
0,0,2023-04-09 23:01:03+00:00,1.645200e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...,neutral
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member,neutral
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...,negative
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...,neutral


In [ ]:
ct2.to_csv('CleanLabeledTweets2.csv', index = False)

In [ ]:
# Load dataset of clean labelled tweets 

data2 = pd.read_csv('/content/CleanLabeledTweets2.csv')
data2.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action
0,0,2023-04-09 23:01:03+00:00,1.645200e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...,neutral
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member,neutral
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...,negative
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...,neutral


In [ ]:
# Assign States
data2['State'] = data2['Retweets'].apply(RT_to_state).shift(-1, fill_value=1) 
data2.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State
0,0,2023-04-09 23:01:03+00:00,1.645200e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral,3
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...,neutral,3
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member,neutral,4
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...,negative,2
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...,neutral,3


In [ ]:
# Check distribution 
data2['State'].value_counts()

2    1717
3    1407
1     892
4     417
Name: State, dtype: int64

In [ ]:
# Function for next state 
# Use the shift function 


data2['Next_State'] = data2['State'].shift(1) # Shifts the states columns but as floats 

data2.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State
0,0,2023-04-09 23:01:03+00:00,1.645200e+18,23,0,0,0,rt w saskatoon shareholder meeting stoplng in...,neutral,3,NaN
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...,neutral,3,3.0
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member,neutral,4,3.0
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...,negative,2,4.0
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...,neutral,3,2.0


In [ ]:
data2 = data2.drop(0)
data2.head()

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...,neutral,3,3.0
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member,neutral,4,3.0
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...,negative,2,4.0
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...,neutral,3,2.0
5,5,2023-04-09 21:15:26+00:00,1.645170e+18,16,0,0,0,rt long treat disposable asked wait ask reaso...,negative,4,3.0


In [ ]:
data2['Reward'] = data2['Next_State'].apply(NextState_to_Reward)
data2.head(10)

,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State,Reward
1,1,2023-04-09 21:26:41+00:00,1.645180e+18,12,0,17,0,new group reasonable student amp youth rise c...,neutral,3,3.0,25
2,2,2023-04-09 21:16:45+00:00,1.645170e+18,18,0,0,0,rt official launch join 100 community member,neutral,4,3.0,25
3,3,2023-04-09 21:16:12+00:00,1.645170e+18,108,0,0,0,rt young people risk arrest politician someth...,negative,2,4.0,100
4,4,2023-04-09 21:16:04+00:00,1.645170e+18,5,0,0,0,rt go big need make even big eye event let u ...,neutral,3,2.0,10
5,5,2023-04-09 21:15:26+00:00,1.645170e+18,16,0,0,0,rt long treat disposable asked wait ask reaso...,negative,4,3.0,25
6,6,2023-04-09 20:07:50+00:00,1.645160e+18,359,0,0,0,rt brand-new youth-led collective 20 day bloc...,neutral,2,4.0,100
7,7,2023-04-09 14:56:56+00:00,1.645080e+18,2,0,0,0,rt,neutral,1,2.0,10
8,8,2023-04-09 14:56:19+00:00,1.645080e+18,1,0,0,0,rt applaud work towards fossilfueldivestment ab,positive,2,1.0,-1
9,9,2023-04-09 00:09:24+00:00,1.644850e+18,2,0,0,0,rt rbc est la ptro-banque du canada elle prte...,neutral,2,2.0,10
10,10,2023-04-09 00:09:13+00:00,1.644850e+18,5,0,0,0,rt le gaz n'est pa une nergie de transition s...,neutral,2,2.0,10


In [ ]:
data_reverse2 = data2[::-1].reset_index(drop=True)
data_reverse2.head()


,Unnamed: 0,Timestamp,User ID,Retweets,Replies,Likes,Quotes,CleanTweet,Action,State,Next_State,Reward
0,4448,2022-05-25 18:36:15+00:00,1.529530e+18,1,0,1,1,`` 100m est un dbut et pas assez pour redress...,neutral,1,1.0,-1
1,4447,2022-05-25 18:38:16+00:00,1.529530e+18,0,0,0,0,`` aujourd'hui 10 officiers de gisci ont enva...,neutral,1,1.0,-1
2,4446,2022-05-25 18:42:13+00:00,1.529530e+18,2,0,0,0,`` bcp de nouveaux visage sur le yintah lors ...,neutral,1,2.0,10
3,4445,2022-05-25 19:04:24+00:00,1.529540e+18,114,0,0,0,rt inspection office wet'suwet'en reveal wate...,negative,2,4.0,100
4,4444,2022-05-25 19:04:31+00:00,1.529540e+18,10,0,0,0,rt erosion sediment-control issue cost coasta...,negative,4,2.0,10


In [ ]:
# Generate Final dataset 

dataset2 = data_reverse2.drop(columns=['Timestamp',	'Retweets',	'Replies',	'Likes',	'Quotes',	'CleanTweet'])
dataset2.head()

,Unnamed: 0,User ID,Action,State,Next_State,Reward
0,4448,1.529530e+18,neutral,1,1.0,-1
1,4447,1.529530e+18,neutral,1,1.0,-1
2,4446,1.529530e+18,neutral,1,2.0,10
3,4445,1.529540e+18,negative,2,4.0,100
4,4444,1.529540e+18,negative,4,2.0,10


In [ ]:
# Reorder columns 

Dataset_final2 = pd.DataFrame({'State' : dataset2['State'], 'Action' : dataset2['Action'], 'Reward' : dataset2['Reward'], 'Next_State': dataset2['Next_State']})
Dataset_final2.head()

,State,Action,Reward,Next_State
0,1,neutral,-1,1.0
1,1,neutral,-1,1.0
2,1,neutral,10,2.0
3,2,negative,100,4.0
4,4,negative,10,2.0


In [ ]:
# Save Dataset 
Dataset_final2.to_csv('Final_Datasetnew2.csv', index = False)